In [1]:
import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

In [2]:
%load_ext autoreload
%autoreload 2
# Hotfix repo
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from preprocessing import read

In [3]:
# TODO change to your own
datapath = '../../data/'
prfile = 'prAdjust_FR-Metro_MPI-ESM1-2-LR_historical_r1i1p1f1_CNRM_CNRM-ALADIN63-emul-CNRM-UNET11-tP22_v1-r1_MF-CDFt-SAFRAN-1985-2014_day_18500101-20141231.nc'
tasfile = 'tasAdjust_FR-Metro_MPI-ESM1-2-LR_historical_r1i1p1f1_CNRM_CNRM-ALADIN63-emul-CNRM-UNET11-tP22_v1-r1_MF-CDFt-ANASTASIA-SAFRAN-1985-2014_day_18500101-20141231.nc'
mergedfile = 'tas_pr_1972_2100.nc'

labelsfilepath = '../data/incendies/incendies_V3.csv'

### Read

In [10]:
# NOTE Old
#start_date=np.datetime64('2004-01-01')

# Read label data
#gdf = read.read_labels(labelsfilepath)

# Read meteo data
#ds = read.read_nc_files([(datapath+prfile, 'prAdjust'), (datapath+tasfile, 'tasAdjust')], start_date=start_date)

### Data for train/test

In [ ]:
# Read label data
gdf = read.read_labels(labelsfilepath)

ds = xr.open_dataset(datapath+mergedfile, chunks={"time": 100})
ds = ds.rio.write_crs("EPSG:27572") 

start_date, end=np.datetime64('2004-01-01'), np.datetime64('2024-12-31')
ds = ds.sel(time=slice(start_date, end))

# Align labels to data
ds = read.align_data(ds, gdf)

# Extact pixelwise raw timeseries
rawdf = read.extract_pixel_timeseries(ds)

# store 
rawdf.to_parquet("../data/raw_2004_2024.parquet")

In [11]:
# clear memory
del ds, rawdf

### Inference data

In [12]:
# NOTE no label data for the future
ds = xr.open_dataset(datapath+mergedfile, chunks={"time": 100})
ds = ds.rio.write_crs("EPSG:27572") 

start_date, end= np.datetime64('2024-01-01'), np.datetime64('2049-12-31'),
ds = ds.sel(time=slice(start_date, end))

# Extact pixelwise raw timeseries
rawdf = read.extract_pixel_timeseries(ds)

# store 
rawdf.to_parquet("../data/raw_2025_2050.parquet")

KeyError: "['spatial_ref'] not found in axis"

In [13]:
rawdf

NameError: name 'rawdf' is not defined

In [ ]:
# clear memory
del ds, rawdf